In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.6/605.6 kB 10.4 MB/s eta 0:00:00


In [ ]:
# Identify Leader-Follower Pairs (Blue=Leader, Red=Follower) for:
# cars moving right:
def dir_right(box1, boxes, frame, count, leader_follower_data):
  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
        if box1_left-box1_w<box2_right<box1_left: # Checking length zone (width of leader car)
          LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
          LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
          if LG<LT:
            cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
            cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)
            data = [[count, box1.id, box2.id]]
            df = pd.DataFrame(data, columns=['Frame', 'Leader', 'Follower'])

            leader_follower_data = pd.concat([leader_follower_data, df], ignore_index=True)

  return leader_follower_data

# cars moving left:
def dir_left(box1, boxes, frame, count, leader_follower_data):
  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      if (box1_top<box2_top<box1_bot or box1_top<box2_bot<box1_bot) or box1_top<box2_cy<box1_bot: # Checking width zone (height of leader car)
        if box1_right+box1_w>box2_left>box1_right: # Checking length zone (width of leader car)
          LT = 0.5*(box1_h) + box2_h # Calculating Lateral Threshold
          LG = abs(box1_cy - box2_cy) # Calculating Lateral Gap
          if LG<LT:
            cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
            cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)
            data = [[count, box1.id, box2.id]]
            df = pd.DataFrame(data, columns=['Frame', 'Leader', 'Follower'])

            leader_follower_data = pd.concat([leader_follower_data, df], ignore_index=True)

  return leader_follower_data

# cars moving up:
def dir_up(box1, boxes, frame, count, leader_follower_data):
  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
        if box1_bot+box1_h>box2_top>box1_bot: # Checking length zone (height of leader car)
          LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
          LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
          if LG<LT:
            cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
            cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)
            data = [[count, box1.id, box2.id]]
            df = pd.DataFrame(data, columns=['Frame', 'Leader', 'Follower'])

            leader_follower_data = pd.concat([leader_follower_data, df], ignore_index=True)

  return leader_follower_data


# cars moving down:
def dir_down(box1, boxes, frame, count, leader_follower_data):
  for box2 in boxes:
    if box1==box2:
      continue
    else:
      box2_w = box2.xywh[0][2]
      box2_h = box2.xywh[0][3]

      box2_left, box2_top, box2_right, box2_bot = box2.xyxy[0]

      box2_cx = (box2_left+box2_right)*0.5
      box2_cy = (box2_top+box2_bot)*0.5

      if (box1_left<box2_left<box1_right or box1_left<box2_right<box1_right) or box1_left<box2_cx<box1_right: # Checking width zone (width of leader car)
        if box1_top-box1_h<box2_bot<box1_top: # Checking length zone (height of leader car)
          LT = 0.5*(box1_w) + box2_w # Calculating Lateral Threshold
          LG = abs(box1_cx - box2_cx) # Calculating Lateral Gap
          if LG<LT:
            cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (255, 0, 0), 3)
            cv2.rectangle(frame, (int(box2_left), int(box2_top)), (int(box2_right), int(box2_bot)), (0, 0, 255), 3)
            data = [[count, box1.id, box2.id]]
            df = pd.DataFrame(data, columns=['Frame', 'Leader', 'Follower'])

            leader_follower_data = pd.concat([leader_follower_data, df], ignore_index=True)

  return leader_follower_data


In [ ]:
import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
import pandas as pd

# Load the YOLOv8 model
model = YOLO('/content/drive/MyDrive/copy_best.pt')

# Open the video file
video_path = "/content/drive/MyDrive/WictroniX/test_input.mp4"
cap = cv2.VideoCapture(video_path)

# Get input video properties (e.g., frame width, height, and frame rate)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create a VideoWriter object to write the output video
output_video_path = '/content/drive/MyDrive/WictroniX/test_output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Change the codec based on the output format
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

prev_pos={}
current_pos={}
count=0
direction="Stationary"

# Create DataFrame to store leader-follower data
leader_follower_data = pd.DataFrame(columns=['Frame', 'Leader', 'Follower'])

# Loop through the video frames
while cap.isOpened():
  # Read a frame from the video
  success, frame = cap.read()

  if success:
    count += 1  # i.e. at 1 frame, this advances one second
    cap.set(cv2.CAP_PROP_POS_FRAMES, count)
    # Run YOLOv8 inference on the frame
    results = model.track(frame, persist=True)
    boxes = results[0].boxes
    ids = boxes.id

    # Store the current detections(bboxes) with their respective ids in the current_pos list
    current_pos={x:y for x,y in zip(ids.tolist(), boxes.xyxy.tolist())}

    for box1, id in zip(boxes, ids.tolist()):
      box1_w = box1.xywh[0][2]
      box1_h = box1.xywh[0][3]

      box1_left, box1_top, box1_right, box1_bot = box1.xyxy[0]

      box1_cx = (box1_left+box1_right)*0.5
      box1_cy = (box1_top+box1_bot)*0.5

      # Drawing bounding boxes with their ID on the current frame
      cv2.rectangle(frame, (int(box1_left), int(box1_top)), (int(box1_right), int(box1_bot)), (0, 255, 0), 1)
      cv2.putText(
        frame,
        f"Id {id}",
        (int(box1_left), int(box1_top)),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
      )
      if ((id in prev_pos.keys()) and (id in current_pos.keys())):
        x_difference = current_pos[id][0] - prev_pos[id][0]
        y_difference = current_pos[id][1] - prev_pos[id][1]

        # Logic to find the direction:
        # if width>height, car moving horizontally
        if (current_pos[id][2]-current_pos[id][0]) > (current_pos[id][3]-current_pos[id][1]):
          if abs(x_difference) > abs(y_difference):
            if x_difference > 0:
              direction = "Right"
            elif x_difference < 0:
              direction = "Left"
            else:
              direction="Stationary"
          else:
            direction="Stationary"

        # if height>width, car moving vertically
        else:
          if abs(x_difference) < abs(y_difference):
            if y_difference > 0:
              direction = "Down"
            elif y_difference < 0:
              direction = "Up"
            else:
              direction="Stationary"
          else:
              direction="Stationary"
      else:
        direction="Stationary"

      # Writing the direction above the bboxes
      x, y = current_pos[id][0:2]
      cv2.putText(frame, direction, (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

      # Comparing every other bbox with the current bbox to find Leader-Follower Pairs
      if direction=="Right":
        leader_follower_data = dir_right(box1, boxes, frame, count, leader_follower_data)

      elif direction=="Left":
        leader_follower_data = dir_left(box1, boxes, frame, count, leader_follower_data)

      elif direction=="Up":
        leader_follower_data = dir_up(box1, boxes, frame, count, leader_follower_data)

      elif direction=="Down":
        leader_follower_data = dir_down(box1, boxes, frame, count, leader_follower_data)

    prev_pos = current_pos
    if count>=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)):
      break

    # Write the processed frame to the output video
    out.write(frame)

# Save the data to csv
leader_follower_data.to_csv('/content/drive/MyDrive/WictroniX/leader_follower_data.csv', index=False)

cap.release()
out.release()

Streaming output truncated to the last 5000 lines.
0: 384x640 1 Bus, 31 Carss, 1 LCV, 1 MAV, 5 Rickshaws, 1 Truck, 629.3ms
Speed: 6.0ms preprocess, 629.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 30 Carss, 1 LCV, 5 Rickshaws, 2 Trucks, 609.4ms
Speed: 2.4ms preprocess, 609.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 30 Carss, 2 LCVs, 5 Rickshaws, 1 Truck, 626.1ms
Speed: 3.0ms preprocess, 626.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 28 Carss, 4 LCVs, 5 Rickshaws, 1 Truck, 627.3ms
Speed: 2.2ms preprocess, 627.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 28 Carss, 2 LCVs, 5 Rickshaws, 2 Trucks, 622.6ms
Speed: 3.3ms preprocess, 622.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 Bus, 27 Carss, 2 LCVs, 5 Rickshaws, 3 Trucks, 632.8ms
Speed: 2.2ms preprocess, 632.8ms inference, 0.7ms 